# Week 12 Assignment - Neural Networks
## Vivek Golla
## HDS 5230 - 07

In [22]:
import os
import pathlib
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

In [23]:
#creating helper function to preprocess data which is common for all dataset sizes and configurations

# Define metrics
metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.AUC(name='auc')]

#helper function to preprocess data - Read relevant data, initialize X and Y arrays, Scale data, fill missing, and returns X and Y splits.
def load_and_preprocess(path):
    df = pd.read_csv(path)
    df.fillna(0, inplace=True)
    X = df.drop('outcome', axis=1).values
    y = df['outcome'].values
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [24]:
#initialize results list
results = []

#this function fits the deep learning model
#logistic regression classifier using sigmoid activation function for binary classification
def run_experiment(datafile, layers_config, epochs=50, batch_size=32):
    X_train, X_val, y_train, y_val = load_and_preprocess(datafile)

    model = Sequential()
    model.add(Dense(layers_config[0], activation='sigmoid', input_dim=X_train.shape[1]))
    for nodes in layers_config[1:]:
        model.add(Dense(nodes, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=metrics)

    start = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val), verbose=0)
    end = time.time()

    train_error = 1 - history.history['accuracy'][-1]
    val_error = 1 - history.history['val_accuracy'][-1]
    exec_time = end - start
    #adding output to the results list
    results.append({
        "Data Size": datafile.split('_')[1].split('.')[0],
        "Layers": f"{len(layers_config)} x {layers_config[0]}",
        "Train Error": round(train_error, 4),
        "Validation Error": round(val_error, 4),
        "Execution Time (s)": round(exec_time, 2)
    })

In [25]:
run_experiment("synthetic_1000.csv", [4])
run_experiment("synthetic_10000.csv", [4])
run_experiment("synthetic_100000.csv", [4], batch_size=64)

run_experiment("synthetic_1000.csv", [4, 4])
run_experiment("synthetic_10000.csv", [4, 4])
run_experiment("synthetic_100000.csv", [4, 4], batch_size=64)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

In [26]:
results_df = pd.DataFrame(results)
print(results_df)

  Data Size Layers  Train Error  Validation Error  Execution Time (s)
0      1000  1 x 4       0.0512            0.0750               11.73
1     10000  1 x 4       0.0015            0.0010               37.96
2    100000  1 x 4       0.0008            0.0009              221.84
3      1000  2 x 4       0.1187            0.1400               13.80
4     10000  2 x 4       0.0016            0.0015               42.68
5    100000  2 x 4       0.0008            0.0013              217.98
